# 函数
本文是《流畅的Python》第五章的笔记。

## 一等对象
在Python中函数是一等对象。所谓的一等对象就是满足下列条件的程序实体：
- 在运行时创建
- 能赋值给变量或数据结构中的元素
- 能作为参数传递给函数
- 能作为函数的返回结果

在Python中，整数、字符串、列表、字典等等内置数据结构都是一等对象。

## 高阶函数
接收函数作为参数，或者把函数作为返回结果的函数是高阶函数(higher-order function)。比如内置的map、sorted、filter等函数都是高阶函数。

### map
`map(func, iterable)`

map函数接收两个参数，一个是函数名字，一个是可迭代的对象。map将参数中的函数分别作用于可迭代对象的每一个元素上，并返回一个map对象。通过`list`或者`tuple`方法将其序列化即可得到结果序列。

In [8]:
def triple(num):
    return num * 3
print(map(triple, [1, 2, 3]))
print(list(map(triple, [1, 2, 3])))

[3, 6, 9]


### reduce
`reduce(func, iterable)`

reduce函数同样接收一个函数和一个可迭代对象作为参数。reduce将函数作用于序列的元素上，并把结果继续和下一个元素做运算，最终规约为一个值。由此，传递给reduce的函数必须接收两个参数：使用两个元素做第一次运算，再将结果作为一个元素和下一个元素继续运算……

**reduce在Python2中是内置函数，但是Python3将reduce移到了functools模块中，使用前必须先导入**

In [11]:
from functools import reduce
def add_a_b(a, b):
    return a + b

reduce(add_a_b, [1,2,3,4,5])

15

### filter
filter函数用于过滤序列。filter函数参数和上述两个函数一模一样，只是filter将传入的函数作为条件，根据返回值为True还是False判断是否保留序列中的元素。filter函数返回值也是一个filter对象，需要将其序列化才能看到值的情况。

In [17]:
list(filter(lambda x: x % 2 == 0, range(1,11)))

[2, 4, 6, 8, 10]

### 匿名函数
上边的一行表达式除了filter还用到了匿名函数lambda。Python对匿名函数有较多的限制，只能使用纯表达式。如在匿名函数中不能赋值，不能使用while等语句。除此之外，匿名函数就是一个没有名称的函数，没什么特别。当一个函数不需要显示定义被多次重用时，匿名函数更加方便。

上述的`lambda x: x % 2 == 0`其实就相当于：
```python
def foo(x):
   return x % 2 == 0
```
除了作为参数传递给高阶函数外，Python很少使用匿名函数。事实上，因为匿名函数的可读性不高，我们也应该尽量避免使用匿名函数。

利用Python的列表推导和生成器表达式可以替换大多数的高阶函数与匿名函数结合的代码。比如过滤1-10中的偶数：

In [23]:
[n for n in range(1,11) if n%2 == 0]

[2, 4, 6, 8, 10]

## 函数参数

在代码中我们最常见到的是*定位参数*，也就是靠传入实参的位置来分别给形参赋值。在Python3中，Python新增了*仅限关键字参数(keyword-only argument)*。
顾名思义，就是只能通过指定关键字传值的参数。

In [21]:
def f(a, *, b):
    return a + b

如果还是正常的传入参数，Python会报错`TypeError: f() takes 1 positional argument but 2 were given`。这里的`b`就是关键字参数，它必须显式的指定才能够传入值，否则不管传入几个值，第二个及以后的都会被`*`接收。

In [20]:
f(1, b=3)

4

那如果想要指定a,b都是仅限关键字参数呢？很简单，把`*`放在最前面就可以了。`def f(*, a, b)`

### 函数注解
Python3提供了对函数注解的语法，用来在函数声明中指明函数的参数类型和返回值类型。

```python
def demo(text, number):
    result = text + str(number)
    return result
```
注解后的版本：

In [3]:
def demo_note(text:str, number:"int > 0"=100) -> str:
    result = text + str(number)
    return result

函数声明的各个参数可以用冒号(:)来添加注解。如果参数有默认值的话，注解放在参数名和=之间。返回值的注解是在函数声明末尾和：之间添加->表达式。注解一般用类型(int或者str)或者字符串(int > 0)。

函数注解并不会做任何处理，只是把注解存储在函数的`__annotations__`属性中。

In [4]:
demo_note.__annotations__

{'number': 'int > 0', 'return': str, 'text': str}

### 提取函数信息
函数有一个`__default__`属性，保存着函数的定位参数和关键字参数的默认值。仅限关键字参数在`__kwdefaults__`属性中。参数的名称在`__code__`中。

## operator和functools模块
operator模块提供了多个运算符对应的函数，比如`+`对应`add()`函数、`+=`对应`iadd`函数……利用这些函数可以进行写出类似于函数编程风格的代码。

### operator.itemgetter
除了运算符函数，operator模块中还有一类函数，可以从序列中读取元素或者读取元素对象属性。因此，`itemgetter`和`attrgetter`函数其实可以根据我们传递的值自行构建函数。

In [26]:
# 利用itemgetter对元组列表排序
from operator import itemgetter

data = [('hello', 'jp', (35, 25)), ('world', 'zh', (55, 88))]
one = itemgetter(1)
print(one(data))

print(sorted(data, key=one, reverse=True))

('world', 'zh', (55, 88))
[('world', 'zh', (55, 88)), ('hello', 'jp', (35, 25))]


stackoverflow上有个问题是[如何用字典排序](https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value)。排名第一的答案给出的结果就是利用`operator.itemgetter`函数：
```python
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
```

### functools.partial
functools模块提供了一系列的高阶函数，其中functools.partial函数的作用是“冻结”函数参数。它基于一个函数创建一个新的可调用对象，把原函数的某些参数固定，这样改编后的函数所需要的参数更少。

In [28]:
from operator import mul
from functools import partial

#重写triple函数
triple = partial(mul, 3)
triple(7)

21